In [ ]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import growthepieapi_utils as gtp
import l2beat_utils as ltwo
sys.path.pop()

# # Usage
merged_df = gtp.get_growthepie_api_data()
merged_df = merged_df.reset_index()
print(merged_df.sample(5))

l2beat_df = ltwo.get_l2beat_activity_data()
print(l2beat_df.sample(5))

In [ ]:
# Post to Dune API
d.write_dune_api_from_pandas(merged_df, 'growthepie_l2_activity',\
                             'L2 Usage Activity from GrowThePie')
d.write_dune_api_from_pandas(l2beat_df, 'l2beat_l2_activity',\
                             'L2 Usage Activity from L2Beat')